In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [814 kB]
Err:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
  File has unexpected size (815853 != 814314). Mirror sync in progress? [IP: 152.195.19.142 443]
  Hashes of expected file:
   - Filesize:814314 [weak]
   - SHA256:257071ac3a46f8e8ba340c2bd6b88466f

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-07-12 06:10:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-07-12 06:10:12 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GamingCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [5]:
df = df.dropna()
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [6]:
df.count()

145425

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# Transform DataFrame to fit review_id table

In [8]:
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
review_id_df = review_id_df.dropDuplicates(["review_id"])

In [10]:
from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



# Transform DataFrame to fit products table

In [11]:
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B013PURRZW|Madden NFL 16 - X...|
|B00F4CEHNK| Xbox Live Gift Card|
|B00DNHLFQA|Command & Conquer...|
|B004RMK5QG|Playstation Plus ...|
|B00G9BNLQE|Saints Row IV - E...|
+----------+--------------------+
only showing top 5 rows



In [12]:
products_df = products_df.dropDuplicates(["product_id"])

In [13]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



# Transform DataFrame to fit customers table

In [14]:
customers_df = df.select(["customer_id"])
customers_df.show(5)

+-----------+
|customer_id|
+-----------+
|   21269168|
|     133437|
|   45765011|
|     113118|
|   22151364|
+-----------+
only showing top 5 rows



In [15]:
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   40550841|    1|
|    1136652|    1|
|   36526525|    1|
|     564588|    1|
|    2830122|    1|
+-----------+-----+
only showing top 5 rows



In [16]:
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40550841|             1|
|    1136652|             1|
|   36526525|             1|
|     564588|             1|
|    2830122|             1|
+-----------+--------------+
only showing top 5 rows



In [17]:
customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df["customer_count"].cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



# Transform DataFrame to fit vine table

In [18]:
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSH1OZ87OYK92|          2|            2|          3|   N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|
| R3YOOS71KM5M9|          5|            0|          0|   N|
|R3R14UATT3OUFU|          5|            0|          0|   N|
| RV2W9SGDNQA2C|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [19]:
vine_df = vine_df.dropDuplicates(["review_id"])

In [20]:
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))

vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Write DataFrames to Database

In [21]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cww47e97r82y.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)